In [1]:
import re
from bs4 import BeautifulSoup
import requests
from pathlib import Path
from bs4.element import Comment

In [2]:
pwd

'/home/duhan/source/django_mainframe'

In [3]:
admin_lte_home = Path('AdminLTE')
pages_home = admin_lte_home.joinpath('dist/pages')
template_home = Path('mainframe/templates/mainframe')
static_home = Path('mainframe/static/mainframe')
# adminlte_home = static_home.joinpath(static_home)

In [4]:
def extract_resource(url):
    """ """
    path = pages_home.joinpath(url)
    if path.exists(): 
        new_path = static_home.joinpath(url.replace('../../', ''))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(path.read_bytes())        
    else:
        new_path = static_home.joinpath(re.sub('https://.*/npm/@*', '', url).replace('@', '/'))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(requests.get(url).content)
    print(new_path.as_posix())
    return new_path.as_posix()

In [5]:
def solve_resource(filename):
    with open(filename, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')    
    attrname_dict = {'link': 'href', 'script': 'src', 'img': 'src'}
    for tag in soup.find_all(['script', 'link', 'img']):
        attrname = attrname_dict.get(tag.name, None)
        if not attrname: continue
        url = tag.get(attrname)
        if not url: continue
        print(url)
        tag[attrname] = extract_resource(url) 
    return soup.prettify()

In [6]:
original_html = solve_resource(
    pages_home.joinpath('index.html').as_posix())

https://cdn.jsdelivr.net/npm/@fontsource/source-sans-3@5.0.12/index.css
mainframe/static/mainframe/fontsource/source-sans-3/5.0.12/index.css
https://cdn.jsdelivr.net/npm/overlayscrollbars@2.3.0/styles/overlayscrollbars.min.css
mainframe/static/mainframe/overlayscrollbars/2.3.0/styles/overlayscrollbars.min.css
https://cdn.jsdelivr.net/npm/bootstrap-icons@1.11.0/font/bootstrap-icons.min.css
mainframe/static/mainframe/bootstrap-icons/1.11.0/font/bootstrap-icons.min.css
../../dist/css/adminlte.css
mainframe/static/mainframe/dist/css/adminlte.css
https://cdn.jsdelivr.net/npm/apexcharts@3.37.1/dist/apexcharts.css
mainframe/static/mainframe/apexcharts/3.37.1/dist/apexcharts.css
https://cdn.jsdelivr.net/npm/jsvectormap@1.5.3/dist/css/jsvectormap.min.css
mainframe/static/mainframe/jsvectormap/1.5.3/dist/css/jsvectormap.min.css
../../dist/assets/img/user1-128x128.jpg
mainframe/static/mainframe/dist/assets/img/user1-128x128.jpg
../../dist/assets/img/user8-128x128.jpg
mainframe/static/mainframe/di

In [7]:
soup = BeautifulSoup(original_html, 'html.parser')

# Clear menu
sidebar_menu = soup.find(attrs={'class': 'sidebar-menu'})
li_list = []
for i, li in enumerate(sidebar_menu.find_all('li', recursive=False)):
    if i in [0, 1, 6, 7]:
        continue
    li.replace_with('')

# Clear content
soup.find(attrs={'class': 'app-content'}).replace_with('')

# Toggle switch
soup.find(attrs={'data-lte-toggle': 'sidebar'}).find('i').replace_with('🌐')

# Remove useless script
for script in soup.find_all('script'):
    src = script.attrs.get('src')
    if src is not None:
        if any([k in src for k in ['adminlte', 'bootstrap']]):
            continue    
    script.replace_with('')  

# Remove useless css
for script in soup.find_all('link'):
    href = script.attrs.get('href')
    if href is not None:
        if any([k in href for k in ['adminlte', 'bootstrap']]):
            continue    
    script.replace_with('')  

# Clear all comments
for commnet in soup.find_all(string=lambda text: isinstance(text, Comment)):
    commnet.replace_with('')

# Delete meta
for meta in soup.find_all('meta')[2:]:
    meta.replace_with('')

# Replace title
soup.find('title').string='数据中台管理'

path = template_home.joinpath('main.html')
path.parent.mkdir(exist_ok=True, parents=True)
path.write_text(soup.prettify())

13756